In [1]:
import time
from datetime import datetime

import pandas as pd
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [2]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
date_fr = "11/01/2018"
date_to = "12/31/2021"

In [4]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [5]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 2).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [6]:
records = pd.concat(dfs, axis=0, ignore_index=True)

In [7]:
if records is not None and not records.empty:
    chunk_size = 1000
    for i in range(0, len(records), chunk_size):
        chunk = records.iloc[i : i + chunk_size]

        stmt = insert(reservation).values(chunk.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)
else: 
    print("empty")

In [8]:
records.shape

(491304, 36)

In [9]:
records.isna().sum()

ref_id                           0
res_id                           0
hotel_id                         0
operator_id                      0
operator_code                    0
bkg_ref                          0
guest_name                     340
sales_date                       0
in_date                          0
out_date                         0
room_type                     4832
room_code                    14300
meal                             0
days                             0
adult                            0
child                            0
purchase_price                   0
purchase_currency            11271
sales_price                      0
sales_currency               12567
purchase_price_indicator       989
sales_price_indicator         1555
create_date                      0
last_modified_date               0
cancellation_date           327597
status                           0
status4                          0
status5                          0
purchase_contract_id